In [2]:
from __future__ import division
from sklearn.preprocessing import CategoricalEncoder
from sklearn.preprocessing import LabelEncoder

import torchvision.models as models
from multiprocessing import Pool
import time
import numpy as np
import pandas as pd
import datetime as dt
import os.path as dpath
import matplotlib.pyplot as plt
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
import pdb
from pathlib import Path
import torch.optim as optim


import torch.nn as nn
import torch.nn.functional as F

from IPython.display import FileLink, FileLinks


%matplotlib inline

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)


In [10]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.LongTensor([word_to_ix["hello"]])
hello_embed = embeds(Variable(lookup_tensor))
print(hello_embed)

Variable containing:
-0.4194  2.4065  0.0318  0.2678  0.8435
[torch.FloatTensor of size 1x5]



In [ ]:
cat_sz = [(c, len(joined_samp[c].cat.categories)+1) for c in cat_vars]

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [50]:
path = "/home/edwin/Datasets/competitions/avito-demand-prediction/"
PATH = Path(path)
TRN_CSV = pd.read_csv(PATH/'train.csv')
TRN_CSV["city"] = TRN_CSV["city"].astype("category").cat.as_ordered()
train, valid = train_test_split(TRN_CSV, train_size=10, test_size=4)
sample = train.copy()

In [51]:
len(train)

10

In [14]:
TRN_CSV["city"] = TRN_CSV["city"].astype("category").cat.as_ordered()

In [52]:
x = sample["city"].cat.codes.values

In [21]:
c = len(TRN_CSV["city"].cat.categories)+1 

In [30]:
mc = min(50, (c+1)//2)

In [24]:
e1 = nn.Embedding(c, 50)

In [29]:
len(e1(Variable(torch.LongTensor([1276]))).view((-1,1)))

50

In [36]:
hidden_size = 100

In [164]:
class EmbeddingModel(nn.Module):
    def __init__(self):
        super(EmbeddingModel, self).__init__()
        self.embed = nn.Embedding(c, mc)
        self.linear1 = nn.Linear(mc, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        embed = self.embed(x).view((-1,mc))
        out = self.linear1(embed)
        out = self.relu(out)
        out = self.linear2(out)
        return out
        

In [165]:
model = EmbeddingModel()

In [166]:
model

EmbeddingModel(
  (embed): Embedding(1734, 50)
  (linear1): Linear(in_features=50, out_features=100, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=100, out_features=1, bias=True)
)

In [167]:
x

array([1632, 1424, 1048, 1632,  231,  949,  585, 1023, 1494, 1274], dtype=int16)

In [168]:
x_train = np.array(x, dtype="int")

In [169]:
len(x_train)

10

In [170]:
x_train

array([1632, 1424, 1048, 1632,  231,  949,  585, 1023, 1494, 1274])

In [171]:
y = sample["deal_probability"].values

In [172]:
y

array([ 0.76874,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.80323,  0.     ,  0.86521])

In [173]:
y_train = np.array(y, dtype="float32")

In [174]:
y_train

array([ 0.76874,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.80323,  0.     ,  0.86521], dtype=float32)

In [175]:
x_train = x_train.reshape(-1,1)
y_train = y_train.reshape(-1,1)

In [176]:
x_train.shape, y_train.shape

((10, 1), (10, 1))

In [177]:
x_ = Variable(torch.from_numpy(x_train))
y_ = Variable(torch.from_numpy(y_train))

In [178]:
x_, y_

(Variable containing:
  1632
  1424
  1048
  1632
   231
   949
   585
  1023
  1494
  1274
 [torch.LongTensor of size 10x1], Variable containing:
  0.7687
  0.0000
  0.0000
  0.0000
  0.0000
  0.0000
  0.0000
  0.8032
  0.0000
  0.8652
 [torch.FloatTensor of size 10x1])

In [179]:
out = model(x_)

In [180]:
out

Variable containing:
-0.5251
 0.0773
-0.2549
-0.5251
-0.2377
 0.0286
-0.2618
-0.1076
-0.1645
-0.3047
[torch.FloatTensor of size 10x1]

In [3]:
class DualEmbeddingModel(nn.Module):
    def __init__(self):
        super(DualEmbeddingModel).__init__()
    
    def forward(self, x_cat):
        return ""
    
    

In [7]:
# list comprehension

In [22]:
mods = nn.ModuleList([nn.Embedding(i+1, 2) for i in range(10)])

In [25]:
mods

ModuleList(
  (0): Embedding(1, 2)
  (1): Embedding(2, 2)
  (2): Embedding(3, 2)
  (3): Embedding(4, 2)
  (4): Embedding(5, 2)
  (5): Embedding(6, 2)
  (6): Embedding(7, 2)
  (7): Embedding(8, 2)
  (8): Embedding(9, 2)
  (9): Embedding(10, 2)
)

In [23]:
e1 = nn.Embedding(1, 2)

In [24]:
e1(Variable(torch.LongTensor([0])))

Variable containing:
-0.7655 -1.0287
[torch.FloatTensor of size 1x2]

In [15]:
e1(Variable(torch.from_numpy(np.array([1]))))

RuntimeError: index out of range at /opt/conda/conda-bld/pytorch_1522182087074/work/torch/lib/TH/generic/THTensorMath.c:277

In [28]:
mm = [e(Variable(torch.from_numpy(np.array([0])))) for i, e in enumerate(mods)]

In [31]:
mm

[Variable containing:
 -1.9806  0.5014
 [torch.FloatTensor of size 1x2], Variable containing:
  0.2147 -0.6694
 [torch.FloatTensor of size 1x2], Variable containing:
  0.7822  1.6266
 [torch.FloatTensor of size 1x2], Variable containing:
 -1.3851 -0.6814
 [torch.FloatTensor of size 1x2], Variable containing:
  0.1913  1.0114
 [torch.FloatTensor of size 1x2], Variable containing:
  0.0295  0.5168
 [torch.FloatTensor of size 1x2], Variable containing:
  1.4165 -0.8308
 [torch.FloatTensor of size 1x2], Variable containing:
  0.4237 -2.1189
 [torch.FloatTensor of size 1x2], Variable containing:
  0.2657  0.1312
 [torch.FloatTensor of size 1x2], Variable containing:
  0.4532  0.9231
 [torch.FloatTensor of size 1x2]]

In [32]:
x = torch.cat(mm, 1)

In [34]:
x.shape

torch.Size([1, 20])